In [ ]:
#| default_exp metrics/det_metrics

In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
#| export 
import torch 
import fastcore.all as fc
import numpy as np
from scipy.optimize import linear_sum_assignment
from typing import Tuple , List
import torchmetrics

from voxdet.bbox_func.bbox_iou import calculate_iou

## Assign TP-FP-FN

- When comparing a ground truth (gt) bounding box with a predicted bounding box, the following terms are used to evaluate the performance of the prediction:
- `True Positive (tp):` a predicted bounding box that correctly identifies an object in the image and has a high overlap (usually measured by the Intersection over Union metric) with the corresponding ground truth bounding box.
- `False Positive (fp):` a predicted bounding box that falsely identifies an object in the image and has a low overlap with any ground truth bounding box or no matching ground truth bounding box
- `False Negative (fn):` a ground truth bounding box that is not matched to any predicted bounding box or has a low overlap with any predicted bounding box.
- To assign tp, fp, fn to gt box and pred box, you can use a matching algorithm such as Hungarian algorithm to match each ground truth bounding box with the most similar predicted bounding box based on the overlap.

In [3]:
#| export
def assign_tp_fp_fn_linear_assignment(pred_bbox: np.ndarray, gt_bbox: np.ndarray, iou_thr: float = 0.1):
    """
    :param pred_bbox: predicted bounding boxes
    :param gt_bbox: ground truth bounding boxes
    :param iou_thr: iou threshold used to define true positive
    :return: tp, fp, fn numpy ndarray of true positives, false positives, false negatives respectively

    This function assigns true positives (tp), false positives (fp), and false negatives (fn) to the ground truth and predicted bounding boxes. It uses Hungarian algorithm to find the matching between gt_boxes and pred_boxes based on the iou (intersection over union) score. For each matching, if the iou score is greater than the threshold, it assigns 1 to the tp array of that index and 0 to the fp and fn array of that index. If there is no match or iou is less than threshold it assigns 1 to the fp and fn array of that index.
    """
    # both gt_boxes and pred_boxes are empty, there should be no tp, fp, or fn
    if len(gt_bbox) == 0 and len(pred_bbox) == 0:
        return np.zeros(0), np.zeros(0), np.zeros(0), np.zeros(0)
    # gt_boxes is empty, all predicted bounding boxes should be considered as fp
    if len(gt_bbox) == 0:
        return np.zeros(len(pred_bbox)), np.ones(len(pred_bbox)), np.zeros(0), np.zeros(0)
    # pred_boxes is empty, all ground truth bounding boxes should be considered as fn
    if len(pred_bbox) == 0:
        return np.zeros(0), np.zeros(0), np.ones(len(gt_bbox)), np.zeros(0)

    # calculate iou betwen pred_box and gt_box
    overlaps = calculate_iou(pred_bbox, gt_bbox)
    # use Hungarian algorithm to find the matching between gt_boxes and pred_boxes
    row_ind, col_ind = linear_sum_assignment(-overlaps)

    ## Assign TP, FP, FN
    assignment = np.column_stack([row_ind, col_ind])
    tp, fp, fn, pred_iou = np.zeros(len(pred_bbox)), np.ones(len(pred_bbox)), np.ones(len(gt_bbox)), np.zeros(len(pred_bbox))
    for i, j in assignment:
        iou = overlaps[i, j]
        if iou >= iou_thr:
            tp[i] = 1
            fp[i] = 0
            fn[j] = 0
            pred_iou[i] = iou
    
    tp_iou = pred_iou[tp.astype(np.int8) == 1]
    return tp, fp, fn, tp_iou


In [4]:
x = torch.hstack([torch.randint(20, size=(1000, 1)) for _ in range(3)])
y = torch.Tensor([[40, 40, 40] for i in range(1000)])
xy = torch.hstack([x, y])
xy.shape

torch.Size([1000, 6])

In [5]:
%time _ = assign_tp_fp_fn_linear_assignment(xy.numpy(), xy.numpy())

CPU times: user 42.6 ms, sys: 6.65 ms, total: 49.2 ms
Wall time: 57.1 ms


### Test 1
> Gt boxes are zero

In [6]:
pred_bbox = np.array([[10, 10, 20, 20], [50, 50, 60, 60], [30, 30, 40, 40]])
gt_bbox = np.array([])
TP, FP, FN, TP_IOU = assign_tp_fp_fn_linear_assignment(pred_bbox, gt_bbox)
fc.all_equal(FN, np.array([]))
fc.all_equal(FP, np.array([1, 1, 1]))
fc.all_equal(TP, np.array([0, 0, 0]))
fc.all_equal(TP_IOU, np.array([]))

True

### Test 2
> both gt and pred bboxes are zero

In [7]:
pred_bbox, gt_bbox = np.array([]), np.array([])
TP, FP, FN, TP_IOU = assign_tp_fp_fn_linear_assignment(pred_bbox, gt_bbox)
fc.all_equal(FN, np.array([]))
fc.all_equal(FP, np.array([]))
fc.all_equal(TP, np.array([]))
fc.all_equal(TP_IOU, np.array([]))

True

### Test 3
> no gt boxes but pred boxes are present

In [8]:
pred_bbox = np.array([])
gt_bbox = np.array([[10, 10, 20, 20], [50, 50, 60, 60], [30, 30, 40, 40]])
TP, FP, FN, TP_IOU = assign_tp_fp_fn_linear_assignment(pred_bbox, gt_bbox)
fc.all_equal(TP, np.array([]))
fc.all_equal(FP, np.array([]))
fc.all_equal(FN, np.array([1, 1, 1]))
fc.all_equal(TP_IOU, np.array([]))

True

### Test 4
> Gt boxes and Pred boxes with one overlap exactly

In [9]:
gt_boxes = np.array([(10, 20, 30, 40), (50, 60, 70, 80), (90, 100, 110, 120)])
pred_boxes = np.array([(15, 25, 35, 45), (55, 65, 75, 85), (95, 105, 115, 125)])
TP, FP, FN, TP_IOU = assign_tp_fp_fn_linear_assignment(pred_boxes, gt_boxes)
fc.test_eq(len(FN), len(gt_boxes))
assert len(FP) == len(TP) == len(pred_boxes)
fc.all_equal(TP, np.array([1, 1, 1]))
fc.all_equal(FP, np.array([0, 0, 0]))
fc.all_equal(FN, np.array([0, 0, 0]))

True

In [10]:
TP_IOU

array([0.39130435, 0.39130435, 0.39130435])

In [11]:
AVG_TP_IOU = 0 if not TP_IOU.size else np.mean(TP_IOU, axis=0)
AVG_TP_IOU

0.391304347826087

### Test 5
> One Gt box - multiple Preds with same overlaps

In [12]:
gt_boxes = np.array([(10, 20, 30, 40), (50, 60, 70, 80), (90, 100, 110, 120)])
pred_boxes = np.array([(15, 25, 35, 45), (55, 65, 75, 85), (95, 105, 115, 125), (95, 105, 115, 130)])
TP, FP, FN, TP_IOU = assign_tp_fp_fn_linear_assignment(pred_boxes, gt_boxes)
assert (len(FN) == len(gt_boxes)) & \
       (len(FP) == len(TP) == len(pred_boxes)), \
       "lengths of TP, FP, FN doesnt match with gt_boxes and pred_boxes"

fc.all_equal(TP, np.array([1, 1, 1, 0]))
fc.all_equal(FP, np.array([0, 0, 0, 1]))
fc.all_equal(FN, np.array([0, 0, 0]))

True

### Test-6
>  One Gt box - multiple Preds with different overlap scores

In [13]:
gt_boxes = np.array([(10, 20, 30, 40), (50, 60, 70, 80), (90, 100, 110, 120)])
pred_boxes = np.array([(15, 25, 35, 45), (55, 65, 75, 85), (95, 105, 115, 125), (90, 100, 110, 120)])
TP, FP, FN, TP_IOU = assign_tp_fp_fn_linear_assignment(pred_boxes, gt_boxes)
assert (len(FN) == len(gt_boxes)) & (
        len(FP) == len(TP) == len(pred_boxes)
    ), "lengths of TP, FP, FN doesnt match with gt_boxes and pred_boxes"
fc.all_equal(TP, np.array([1, 1, 0, 1]))
fc.all_equal(FP, np.array([0, 0, 1, 0]))
fc.all_equal(FN, np.array([0, 0, 0]))

True

In [14]:
TP_IOU

array([0.39130435, 0.39130435, 1.        ])

In [15]:
AVG_TP_IOU = 0 if not TP_IOU.size else np.mean(TP_IOU, axis=0)
AVG_TP_IOU

0.5942028985507246

### Test-7
> No overlaps between GT and Preds

In [16]:
gt_boxes = np.array([(10, 20, 30, 40), (50, 60, 70, 80), (90, 100, 110, 120)])
pred_boxes = np.array([(1150, 1125, 1135, 1145), \
                       (1155, 1165, 1175, 1185), \
                       (1115, 1105, 1125, 1125), \
                       (1195, 1105, 2115, 2130)])
TP, FP, FN, TP_IOU = assign_tp_fp_fn_linear_assignment(pred_boxes, gt_boxes)
assert (len(FN) == len(gt_boxes)) & (
        len(FP) == len(TP) == len(pred_boxes)
    ), "lengths of TP, FP, FN doesnt match with gt_boxes and pred_boxes"

fc.all_equal(TP, np.array([1, 1, 1, 1]))
fc.all_equal(FP, np.array([0, 0, 0, 0]))
fc.all_equal(FN, np.array([1, 1, 1]))

True

In [17]:
TP_IOU

array([], dtype=float64)

In [18]:
AVG_TP_IOU = 0 if not TP_IOU.size else np.mean(TP_IOU, axis=0)
AVG_TP_IOU

0

### Test-8
> Gt boxes and Pred boxes with one overlap exactly but iou less

In [19]:
gt_boxes = np.array([(10, 20, 30, 40), (50, 60, 70, 80), (90, 100, 110, 120)])
pred_boxes = np.array([(15, 25, 35, 45), (55, 65, 75, 85), (95, 105, 115, 125)])
TP, FP, FN, _ = assign_tp_fp_fn_linear_assignment(pred_boxes, gt_boxes, iou_thr=0.5)
assert (len(FN) == len(gt_boxes)) & (
    len(FP) == len(TP) == len(pred_boxes)
), "lengths of TP, FP, FN doesnt match with gt_boxes and pred_boxes"

fc.all_equal(TP, np.array([0, 0, 0]))
fc.all_equal(FP, np.array([1, 1, 1]))
fc.all_equal(FN, np.array([1, 1, 1]))

True

## Calculate Metrics 

In [30]:
class DetMetrics:
    # inspired from https://github.com/rafaelpadilla/Object-Detection-Metrics#average-precision
    def __init__(self, iou_thr: float = 0.1, conf_thr: float = 0.1, froc_thresholds: Tuple = (0.25, 0.5, 1, 2, 4, 8)):
        fc.store_attr()
        self.reset()
        self._metric_list = ["AP_interp", "FROC_interp", "FROC", "AP", "recall", "tp", "fp", "fn", "precision", "FROC_thresholds", "avg_tp_iou"]

    def update(self, pred_bbox: np.ndarray, pred_scores: np.ndarray, gt_bbox: np.ndarray):
        keep = pred_scores >= self.conf_thr
        pred_bbox = pred_bbox[keep]
        pred_scores = pred_scores[keep]
        tp, fp, fn, tp_iou = assign_tp_fp_fn_linear_assignment(pred_bbox, gt_bbox, iou_thr=self.iou_thr)
        self.num_images += 1
        self.tp = np.hstack([self.tp, tp])
        self.fp = np.hstack([self.fp, fp])
        self.fn = np.hstack([self.fn, fn])
        self.tp_iou = np.hstack([self.tp_iou, tp_iou])
        self.conf = np.hstack([self.conf, pred_scores])

    def reset(self):
        self.tp, self.fp, self.fn, self.conf, self.tp_iou = np.zeros(0), np.zeros(0), np.zeros(0), np.zeros(0), np.zeros(0)
        self.num_images = 0

    def compute(self):
        metrics = {}
        metrics["conf"] = self.conf_thr
        metrics["iou"] = self.iou_thr
        if self.tp.shape[0] == 0:
            for i in self._metric_list: metrics[i] = 0
            metrics["fn"] = np.sum(self.fn)
            return metrics 
        
        tp, fp, _ = zip(*sorted(zip(self.tp, self.fp, self.conf), key=lambda x: x[2], reverse=True))
        precision = np.cumsum(tp) / (np.cumsum(tp) + np.cumsum(fp)+1e-16)
        sensitivity = np.cumsum(tp) / (sum(tp) + sum(self.fn))
        fpr = np.cumsum(fp) / self.num_images

        ## Update metrics
        metrics["FROC_thresholds"] = list(self.froc_thresholds)
        metrics["AP_interp"] = np.interp(np.linspace(0, 1, 11), sensitivity, precision).tolist()
        metrics["FROC_interp"] = np.interp(self.froc_thresholds, fpr, sensitivity).tolist()
        metrics["FROC"] = np.mean(metrics["FROC_interp"])
        metrics["AP"] = np.mean(metrics["AP_interp"])
        metrics["recall"] = sensitivity.max()
        metrics["tp"] = np.sum(self.tp)
        metrics["fp"] = np.sum(self.fp)
        metrics["avg_tp_iou"] = 0 if not self.tp_iou.size else np.mean(self.tp_iou, axis=0)
        metrics["precision"] = metrics["tp"]/(metrics["tp"] + metrics["fp"])
        metrics["fn"] = np.sum(self.fn)
        return metrics

In [20]:
#| export
class DetMetrics(torchmetrics.Metric):
    def __init__(self, 
                 iou_thr: float = 0.1, 
                 conf_thr: float = 0.1, 
                 froc_thresholds: Tuple[float, ...] = (0.25, 0.5, 1, 2, 4, 8), 
                 dist_sync_on_step=False):
        super().__init__(dist_sync_on_step=dist_sync_on_step)
        self.iou_thr = iou_thr
        self.conf_thr = conf_thr
        self.froc_thresholds = froc_thresholds
        self.to_np = lambda x: x.detach().cpu().numpy()

        self.add_state("tps", default=[], dist_reduce_fx=None)
        self.add_state("fps", default=[], dist_reduce_fx=None)
        self.add_state("fns", default=[], dist_reduce_fx=None)
        self.add_state("confs", default=[], dist_reduce_fx=None)
        self.add_state("tp_ious", default=[], dist_reduce_fx=None)
        self.add_state("num_images", default=torch.tensor(0.), dist_reduce_fx=None)

    def update(self, pred_bbox, pred_scores, gt_bbox):
        keep = pred_scores >= self.conf_thr
        pred_bbox = pred_bbox[keep]
        pred_scores = pred_scores[keep]

        tp, fp, fn, tp_iou = assign_tp_fp_fn_linear_assignment(self.to_np(pred_bbox), self.to_np(gt_bbox), self.iou_thr)

        self.tps.append(torch.tensor(tp, dtype=torch.float32).to(self.device))
        self.fps.append(torch.tensor(fp, dtype=torch.float32).to(self.device))
        self.fns.append(torch.tensor(fn, dtype=torch.float32).to(self.device))
        self.tp_ious.append(torch.tensor(tp_iou, dtype=torch.float32).to(self.device))
        self.confs.append(torch.tensor(pred_scores, dtype=torch.float32).to(self.device))
        self.num_images += 1

    def compute(self):
        # Ensure there is at least one TP to avoid division by zero

        self.tps = torch.cat(self.tps)
        self.fps = torch.cat(self.fps)
        self.fns = torch.cat(self.fns)
        self.tp_ious = torch.cat(self.tp_ious)
        self.confs = torch.cat(self.confs)
        self.num_images = torch.sum(self.num_images)
      
        metrics = {}
        metrics["conf"] = self.conf_thr
        metrics["iou"] = self.iou_thr
        
        if self.tps == []:
            metrics["fn"] = self.fns.sum().item()
            # Initialize other metrics as well, if needed
            return metrics 

        # Sorting based on confidences
        sorted_indices = torch.argsort(self.confs, descending=True)
        tp_sorted = self.tps[sorted_indices].cpu().numpy()  # Ensure conversion to numpy
        fp_sorted = self.fps[sorted_indices].cpu().numpy()  # Ensure conversion to numpy

        # Cumulative sums for TP and FP to calculate precision and recall
        tp_cumsum = np.cumsum(tp_sorted)
        fp_cumsum = np.cumsum(fp_sorted)

        num_positives = tp_cumsum[-1] + self.fns.sum().item()

        precision = tp_cumsum / (tp_cumsum + fp_cumsum)
        recall = tp_cumsum / num_positives
        fpr = fp_cumsum / self.num_images.item()

        # Interpolation for AP_interp and FROC_interp
        AP_interp = np.interp(np.linspace(0, 1, 11), recall, precision, right=0)
        FROC_interp = np.interp(self.froc_thresholds, fpr, recall, right=0)

        # Update metrics directly with numpy values
        metrics["FROC_thresholds"] = self.froc_thresholds
        metrics["AP_interp"] = AP_interp.tolist()
        metrics["FROC_interp"] = FROC_interp.tolist()
        metrics["FROC"] = FROC_interp.mean()
        metrics["AP"] = AP_interp.mean()
        metrics["recall"] = recall.max()
        metrics["tp"] = tp_cumsum[-1]
        metrics["fp"] = fp_cumsum[-1]
        metrics["avg_tp_iou"] = np.mean(self.tp_ious.cpu().numpy()) if self.tp_ious.numel() > 0 else 0  # Ensure conversion to numpy and handling as numpy array
        metrics["precision"] = precision[-1]
        metrics["fn"] = self.fns.sum().item()

        return metrics

### Test-1

In [24]:
iou_thr = 0.1
meters = DetMetrics(iou_thr=iou_thr)
meters.tps = torch.Tensor([1, 1, 1, 1, 1])
meters.fps = torch.Tensor([0, 0, 0, 0, 0])
meters.fns = torch.Tensor([0, 0, 0, 0, 0])
meters.confs = torch.Tensor([1, 1, 1, 1, 1])
meters.tp_ious = torch.Tensor([1, 1, 1, 1, 1])
meters.num_images = torch.tensor(3)
meters.eval_thresholds = [1 / 8, 1 / 4, 1, 2, 4, 8]

expected_AP_interp = np.ones(11).tolist()
expected_FROC_interp = np.ones(6).tolist()
metrics = meters.compute()

fc.eq(metrics["AP_interp"], expected_AP_interp)
fc.eq(metrics["FROC_interp"], expected_FROC_interp)
fc.eq(metrics["recall"], 1)
fc.eq(metrics["precision"], 1)
fc.eq(metrics["avg_tp_iou"], 1)

/home/users/shubham.kumar/miniconda3/envs/qct_deep_clone/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: The ``compute`` method of metric DetMetrics was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)


True

In [25]:
iou_thr = 0.3
meters = DetMetrics(iou_thr=iou_thr)
meters.tps = []
meters.fps = torch.Tensor([1, 1, 1, 1, 1])
meters.fns = torch.Tensor([1, 1, 1, 1, 1])
meters.confs = torch.Tensor([0, 0, 0, 0, 0])
meters.tp_ious = []
meters.num_images = torch.tensor(3)
meters.eval_thresholds = [1 / 8, 1 / 4, 1, 2, 4, 8]

expected_AP_interp = np.ones(11).tolist()
expected_FROC_interp = np.ones(6).tolist()
metrics = meters.compute()

fc.eq(metrics["fn"], 5.0)

True

### Test-2

In [26]:
iou_thr = 0.1
meters = DetMetrics(iou_thr=iou_thr)
meters.tps = torch.Tensor([0, 1, 0, 1, 0])
meters.fps = torch.Tensor([1, 0, 1, 0, 1])
meters.fns = torch.Tensor([0, 1, 1, 0])
meters.confs = torch.Tensor([0.5, 0.5, 0.5, 0.5, 0.5])

tp_count = int(meters.tps.sum().item())

torch.manual_seed(24)
meters.tp_ious = torch.rand(tp_count) * (1 - iou_thr) + iou_thr

meters.num_images = torch.tensor(3)
meters.eval_thresholds = [1 / 8, 1 / 4, 1, 2, 4, 8]
metrics = meters.compute()

fc.test_close(metrics["AP_interp"], [0.0, 0.2, 0.4, 0.367, 0.434, 0.4, 0 , 0 , 0 , 0 ,0], eps=1e-2)
fc.test_close(metrics["FROC_interp"], [0.0, 0.25, 0.5, 0.0, 0.0, 0.0], eps=1e-2)
fc.eq(metrics["precision"], 0.4)
fc.eq(metrics["recall"], 0.5)
fc.eq(metrics["fn"], 2)

True

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()